In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import cvxpy as cp ### install cvxpy
#from teamName import getMyPosition 

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))

Loaded 50 instruments for 500 days


In [2]:
# Variable to track days since last position update
days_since_last_update = 0
UPDATE_FREQUENCY = 20  # Update positions every 10 days

##### TODO #########################################
### RENAME THIS FILE TO YOUR TEAM NAME #############
### IMPLEMENT 'getMyPosition' FUNCTION #############
### TO RUN, RUN 'eval.py' ##########################

nInst = 50
currentPos = np.zeros(nInst)

def getMyPosition(prcSoFar):
    global currentPos, days_since_last_update
    
    (nins, nt) = prcSoFar.shape
    if nt < 2:
        return np.zeros(nins)  # Not enough data points to calculate returns
    
    # Calculate daily returns
    returns = np.diff(prcSoFar, axis=1) / prcSoFar[:, :-1]
    
    # Calculate expected returns and covariance matrix
    expected_returns = np.mean(returns, axis=1)
    cov_matrix = np.cov(returns)
    
    # Mean-variance optimization using cvxpy
    w = cp.Variable(nins)
    risk = cp.quad_form(w, cov_matrix)
    ret = expected_returns @ w
    gamma = 3  # Risk aversion parameter

    prob = cp.Problem(cp.Maximize(ret - gamma * risk), 
                      [cp.sum(w) == 1, w >= -1, w <= 1])
    prob.solve()

    optimal_weights = w.value

    # Initialize positions
    new_positions = np.zeros(nins)
    budget = 10000

    # Check if it's time to update positions
    if days_since_last_update == UPDATE_FREQUENCY:
        # Calculate the number of shares for each instrument based on optimal weights
        for i in range(nins):
            current_price = prcSoFar[i, -1]
            if optimal_weights[i] > 0:
                num_shares = (budget * optimal_weights[i]) // current_price
                position_value = num_shares * current_price
                if position_value > dlrPosLimit:
                    num_shares = dlrPosLimit // current_price
            elif optimal_weights[i] < 0:
                num_shares = (budget * optimal_weights[i]) // current_price
                position_value = num_shares * current_price
                if position_value < -dlrPosLimit:
                    num_shares = -dlrPosLimit // current_price
            else:
                num_shares = 0

            new_positions[i] = num_shares
        
        # Reset days_since_last_update
        days_since_last_update = 0
    
    else:
        # Maintain current positions
        new_positions = currentPos
        days_since_last_update += 1
    
    # Update current positions
    currentPos = new_positions
    return currentPos


In [3]:
def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(150, 500):   ### day start to trade at 50 to 500
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        ### this is my modification code for checking position
        #print(newPosOrig) ###check position Number of holding shares
        print(curPrices)
        print(newPos)   ### position number of holding shares after apply limited constraints
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd

[14.56 73.79 47.61 49.71 56.58 11.87 18.02 45.84 68.06 46.75 37.67 27.86
 26.89 50.05 14.13 24.46 35.5  45.4  14.19 31.02 58.53 22.63 67.53 29.79
 63.15 48.03 58.78 29.09 50.85 31.37 13.54 66.72 52.92 40.72 26.01 55.06
 33.15 38.4  21.55 48.95 32.54 53.86 12.95 61.68 35.91 53.81 54.74 33.83
 40.6  54.42]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Day 150 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
[14.62 74.14 47.62 49.73 56.82 11.93 18.   45.83 68.07 46.98 37.82 27.83
 26.82 49.87 14.29 24.15 35.5  45.2  14.2  31.24 58.37 22.56 67.55 29.76
 62.97 48.45 59.29 29.07 50.72 31.43 13.59 66.47 52.97 40.87 26.12 55.05
 32.81 38.84 21.52 48.96 32.6  53.56 12.99 61.03 35.64 53.65 54.68 33.78
 40.54 54.66]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Day 151 value: 0.

In [4]:
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

=====
mean(PL): 35.1
return: 0.00666
StdDev(PL): 349.16
annSharpe(PL): 1.59 
totDvolume: 1842367 
Score: 0.16
